In [1]:
%%spark

import org.apache.spark.sql.delta.DeltaLog
import org.apache.spark.sql.SparkSession
import org.apache.log4j.{Level, Logger}
import io.delta.tables.DeltaTable

// Set logging level to reduce verbosity
Logger.getLogger("org").setLevel(Level.WARN)

// List all databases (lakehouses)
val lakehouses = spark.catalog.listDatabases().collect().find(_.name == "AddressBase")

lakehouses match {
  case Some(lakehouse) =>
    
    // Get all tables in the lakehouse
    val tables = spark.catalog.listTables(lakehouse.name).collect().sortBy(_.name)
    
    tables.foreach { table =>
      try {
        // Delta table path
        val deltaTablePath = s"${lakehouse.locationUri}/${table.name}"
        
        // Create DeltaTable object
        val deltaTable = DeltaTable.forPath(spark, deltaTablePath)
        
        // 1. Perform Checkpoint
        DeltaLog.forTable(spark, deltaTablePath).checkpoint()
        println(s"Checkpoint completed for lakehouse: ${lakehouse.name}, table: ${table.name}")
        
        // 2. Perform Optimization (Compaction)
        spark.sql(s"OPTIMIZE delta.`$deltaTablePath`")
        println(s"Optimization completed for lakehouse: ${lakehouse.name}, table: ${table.name}")
        
        // 3. Disable retention duration check and vacuum
        spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", false)
        deltaTable.vacuum(16)  // vacuum with 16 hours of retention, adjust as per your need
        println(s"Vacuum completed for lakehouse: ${lakehouse.name}, table: ${table.name}")
        
      } catch {
        case e: Exception =>
          // Handle any errors during table maintenance
          println(s"Error processing lakehouse: ${lakehouse.name}, table: ${table.name}. Error: ${e.getMessage}")
      }
    }
    
    // Close the Spark session
    spark.close()

  case None =>
    println("Lakehouse 'AddressBase' not found.")
}


StatementMeta(, 1589276e-4d2d-42ec-9203-a8f92852aae5, 5, Finished, Available, Finished)

Checkpoint completed for lakehouse: AddressBase, table: ab_class_scheme
Optimization completed for lakehouse: AddressBase, table: ab_class_scheme
Deleted 0 files and directories in a total of 1 directories.
Vacuum completed for lakehouse: AddressBase, table: ab_class_scheme
Checkpoint completed for lakehouse: AddressBase, table: abp_blpu
Optimization completed for lakehouse: AddressBase, table: abp_blpu
Deleted 1 files and directories in a total of 1 directories.
Vacuum completed for lakehouse: AddressBase, table: abp_blpu
Checkpoint completed for lakehouse: AddressBase, table: abp_classification
Optimization completed for lakehouse: AddressBase, table: abp_classification
Deleted 1 files and directories in a total of 1 directories.
Vacuum completed for lakehouse: AddressBase, table: abp_classification
Checkpoint completed for lakehouse: AddressBase, table: abp_crossref
Optimization completed for lakehouse: AddressBase, table: abp_crossref
Deleted 2 files and directories in a total of 1